<a href="https://colab.research.google.com/github/mukthar56788/AI-Project/blob/main/Alarm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
from google.colab import files

# Upload the Excel file
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Read Excel starting from the second row (header at row index 1)
df = pd.read_excel(filename, header=3)
df.head()


Saving combine.xlsx to combine (2).xlsx


,SL NO.,CLIENT NAME,CLIENT DOMAIN,EVENT ID,EVENT DAY,EVENT DAY.1,EVENT DOMAIN,SOURCE ID,SOURCE NAME,SOURCE DOMAIN,...,RECURRING,SUGGESTION,ACTIVE,TIME WAITING,ALARM MESSAGE,CONFIGURATION ID,RESOLVED TIME,RESOLVED MESSAGE,VENDORS,DELAY (long)
0,1,KSA,americanaksa,2ef575fc-cab6-309b-a625-4d26b95f38c8,12 May 2025 03:11:35,12 May 2025 00:0:00,americana,5eff9c7e-934d-49f5-a665-6833580caf72,H 5514416 FZ 01,americana,...,NaN,Review the system to design a valid diagnosis ...,True,NaN,Extremely High Freezer Temperature Alarm Occurred,d97e75a8-8da2-4f5c-8806-cd6d1f411027,NaN,NaN,NaN,15.0
1,2,KSA,americanaksa,e506b174-b2cf-3e4e-a8f6-5ca0851e3576,12 May 2025 03:11:25,12 May 2025 00:0:00,americana,15f0332f-aaca-4d2c-8aab-a4a2d4e9a4aa,K 5512422 FZ 01,americana,...,NaN,Review the system to design a valid diagnosis ...,True,NaN,High Freezer Temperature Alarm Occurred,0883d9b7-044a-457c-8f85-664202d876b5,NaN,NaN,NaN,30.0
2,3,KSA,americanaksa,3af207f5-82be-3969-9022-6ee54f17a340,12 May 2025 03:11:05,12 May 2025 00:0:00,americana,89e80ada-bf85-4150-b710-a75bacd1ade1,H 5514133 CH 01,americana,...,NaN,Review the system to design a valid diagnosis ...,True,NaN,Low Chiller Room Temperature Alarm Occurred,aec1f505-962b-43cd-983f-4cfaad2e72a0,NaN,NaN,NaN,30.0
3,4,UAE,americanauae,b9adf3eb-b27a-300b-9040-397558be3c8d,12 May 2025 03:10:02,12 May 2025 00:0:00,americana,6b6f5c16-7d79-44b3-ba19-95a5a254cac6,K 112737 FZ 01,americana,...,NaN,Review the system to design a valid diagnosis ...,True,NaN,Freezer Door is Open,bf423e49-ba90-4476-8deb-a198d2bde4f2,NaN,NaN,NaN,15.0
4,5,UAE,americanauae,d46a6a7a-d410-32b4-98e7-e4a8c3e7215f,12 May 2025 03:8:51,12 May 2025 00:0:00,americana,99f8ce52-1f7e-4922-92e0-7cf2709d02a2,PH 116980 FZ 01,americana,...,NaN,Review the system to design a valid diagnosis ...,True,NaN,Low Freezer Temperature Alarm Occurred,8ec2e275-65f3-4f3d-a597-1b6a1cfc52b3,NaN,NaN,NaN,30.0


In [18]:
# Convert 'EVENT DAY' and 'RESOLVED TIME' columns to datetime
df['EVENT DAY'] = pd.to_datetime(df['EVENT DAY'], errors='coerce')
df['RESOLVED TIME'] = pd.to_datetime(df['RESOLVED TIME'], errors='coerce')

# Combine 'UAE' and 'americanauae' as a single group
df['CLIENT NAME'] = df['CLIENT NAME'].replace({'americanauae': 'UAE'})


In [19]:
results = {}

# 1. Alarm count by SOURCE NAME
source_counts = df['SOURCE NAME'].value_counts().to_dict()
results['alarm_count_by_source_name'] = source_counts

# 2. Alarm count per CLIENT NAME
client_counts = df['CLIENT NAME'].value_counts().to_dict()
results['alarm_count_by_client_name'] = client_counts

# 3. Alarm count by NAME
name_counts = df['NAME'].value_counts().to_dict()
results['alarm_count_by_name'] = name_counts

# 4. Alarm count by CRITICALITY
criticality_counts = df['CRITICALITY'].value_counts().to_dict()
results['alarm_count_by_criticality'] = criticality_counts


In [20]:
# 5. Time taken to resolve in minutes
df['resolution_time_minutes'] = (df['RESOLVED TIME'] - df['EVENT DAY']).dt.total_seconds() / 60

# Optional: Filter valid resolutions
df_resolution = df[['EVENT ID', 'SOURCE NAME', 'resolution_time_minutes']].dropna()

# Add to results (mean time per SOURCE NAME)
resolution_by_source = df_resolution.groupby('SOURCE NAME')['resolution_time_minutes'].mean().round(2).to_dict()
results['average_resolution_time_by_source'] = resolution_by_source


In [28]:
# Ensure SOURCE DOMAIN is available and clean
if 'SOURCE DOMAIN' in df.columns:
    df['Week'] = df['EVENT DAY'].dt.isocalendar().week
    df['Month'] = df['EVENT DAY'].dt.month

    # 6. Weekly alarm count by SOURCE DOMAIN
    weekly_alarms = df.groupby(['CLIENT NAME', 'Week']).size().unstack(fill_value=0)
    results['weekly_alarm_by_region'] = weekly_alarms.to_dict()

    # 7. Monthly alarm count by SOURCE DOMAIN
    monthly_alarms = df.groupby(['CLIENT NAME', 'Month']).size().unstack(fill_value=0)
    results['monthly_alarm_by_region'] = monthly_alarms.to_dict()
else:
    results['weekly_alarm_by_region'] = "SOURCE DOMAIN column not found"
    results['monthly_alarm_by_region'] = "SOURCE DOMAIN column not found"


In [29]:
import json

# Convert all keys and values in results to native Python types
def convert(obj):
    if isinstance(obj, dict):
        return {str(k): convert(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert(i) for i in obj]
    elif hasattr(obj, 'item'):
        return obj.item()
    else:
        return obj

# Convert and save to JSON
with open('alarm_analysis.json', 'w') as f:
    json.dump(convert(results), f, indent=2)

